Importing Libraries

In [1]:
import os
import optuna
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# Set the path to the directories containing your training and testing datasets
train_dir = 'E:\Python\MobileNet-V2-Batik-Image-Classification\dataset-v3[600]-5\data-train'
test_dir = 'E:\Python\MobileNet-V2-Batik-Image-Classification\dataset-v3[600]-5\data-test'

In [3]:
# Set the number of classes in your dataset
num_classes = 5

# Set the input shape for MobileNetV2
input_shape = (224, 224, 3)

# Set the number of trials for hyperparameter tuning
num_trials = 5

Preprocessing and Augmentation

In [4]:
# Create an image data generator with data augmentation for training data
train_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                  rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.4,
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  brightness_range=[0.8, 1.2],
                                  validation_split=0.2)

#note: test data should not have to be augmented
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [5]:
train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape[:2],
                                                    class_mode='categorical',
                                                    subset='training',
                                                    shuffle=True)

Found 435 images belonging to 5 classes.


In [6]:
eval_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                   target_size=input_shape[:2],
                                                   class_mode='categorical',
                                                   subset='validation',
                                                   shuffle=True)

Found 105 images belonging to 5 classes.


In [7]:
test_generator = test_datagen.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape[:2],
                                                  class_mode='categorical',
                                                  shuffle=False)

Found 60 images belonging to 5 classes.
